In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA


##Data Augmentation
- Dia das mães
- Nome da loja
- Feriado

##Algoritmos
- ANN
- XGBoost
- TSAI
- SARIMA



- entrega dia 17/11
- csv com as predicoes
- codigo
- relatorio
    - arquitetura da solução
    - medidas de avaliacao
    - achados e dificuldades
    - licoes aprendidas

In [3]:
def split_data(df):
  web = df[df['tipo_loja'] == 'web']
  fisica = df[df['tipo_loja'] == 'física']

  return fisica, web

def adjust_data(df):
  df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%dT%H:%M:%S.%fZ')
  df = df.pivot(index='ds', columns='loja', values='venda').fillna(0).rename_axis(None, axis=1)

  return df

def scale_data(df):
  for c in df.columns:
    df[c] = MinMaxScaler().fit_transform(df[c].values.reshape(-1, 1)).reshape(1, -1)[0]

  return df

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = pd.concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values


In [ ]:
https://www.kaggle.com/code/robikscube/pt2-time-series-forecasting-with-xgboost/notebook

In [47]:
class TSData:
    def __init__(self, column, df):
        self.name = column
        self.data = df[[column]]
        self.data[column + '_scaled'] = self.__scale_data(self.name, self.data)
        self.features = self.create_features()

    def __str__(self):
        return f'{self.name} - data:{self.data.shape} - features:{self.features.shape}'

    def __scale_data(self, column, df):
        scaler = MinMaxScaler()
        data = scaler.fit_transform(df[column].values.reshape(-1, 1))
        return data.reshape(1, -1)[0]

    def create_features(self):
        features = pd.DataFrame()
        
        features['MA_3'] = self.data[self.name].rolling(3).mean()
        features['MA_7'] = self.data[self.name].rolling(7).mean()
        features['MA_14'] = self.data[self.name].rolling(14).mean()
        features['MA_21'] = self.data[self.name].rolling(21).mean()

        features['STD_3'] = self.data[self.name].rolling(3).std()
        features['STD_7'] = self.data[self.name].rolling(7).std()
        features['STD_14'] = self.data[self.name].rolling(14).std()
        features['STD_21'] = self.data[self.name].rolling(21).std()

        features = features.fillna(0)
        for c in features.columns:
            features[c] = self.__scale_data(c, features)

        return features

In [49]:
#test_file = '/content/drive/MyDrive/PUC/Ciência de Dados e Inteligência Artificial/2022-02/98H05-04 - Aprendizado Supervisionado/TF/data/renner_sales_sample_test.csv'
#train_file = '/content/drive/MyDrive/PUC/Ciência de Dados e Inteligência Artificial/2022-02/98H05-04 - Aprendizado Supervisionado/TF/data/renner_sales_sample_train.csv'

test_file = '../TF/data/renner_sales_sample_test.csv'
train_file = '../TF/data/renner_sales_sample_train.csv'

test = pd.read_csv(test_file)
train = pd.read_csv(train_file)

test = adjust_data(test)
train = adjust_data(train)

train_scaled = scale_data(train.copy(deep=True))
test_scaled = scale_data(test.copy(deep=True))

In [50]:
#dicionario com as series separadas por loja
train_series = {}
for c in train.columns:
    train_series[c] = TSData(c, train)

test_series = {}
for c in test.columns:
    test_series[c] = TSData(c, test)

C:\Users\crist\AppData\Local\Temp\ipykernel_50764\512839091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column + '_scaled'] = self.__scale_data(self.name, self.data)
C:\Users\crist\AppData\Local\Temp\ipykernel_50764\512839091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data[column + '_scaled'] = self.__scale_data(self.name, self.data)
C:\Users\crist\AppData\Local\Temp\ipykernel_50764\512839091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
#tenta prever a venda de uma loja específica criando estimativas para cada semana
#as estimativas devem ser classificadas conforme o range de vendas da loja
#pex:
#range 1: 0-10
#range 2: 10-20
#range 3: 20-30
#e assim por diante
#desse modo o classificador pode ser treinado para prever o range de vendas da loja
#utilizar indicadores de tendência e sazonalidade para estimar o range de vendas
#utilizar o modelo de regressão linear para estimar o range de vendas
#utilizar o modelo de regressão logística para estimar o range de vendas
#utilizar indicadores financeiros para estimar o range de vendas, como por exemplo, 
# média móvel, média ponderada, média exponencial, bandas de bollinger e etc



In [51]:
#itera entre as series de cada loja para treinar o modelo
#for tsc in train_series.values():
tsc = train_series['renner_br_ac_1']
print(tsc)

X = tsc.features
y = tsc.data[tsc.name + '_scaled']

tscv = TimeSeriesSplit(n_splits=10, gap=1)
model = XGBRegressor(learning_rate=.1, booster='gbtree', n_estimators=500)
scores = cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_squared_error')

print(f'RMSE: {np.sqrt(-scores).mean()}')



renner_br_ac_1 - data:(275, 2) - features:(275, 8)
RMSE: 0.1297180142947423


In [52]:
tsc = train_series['renner_br_ac_1']
print(tsc)

X = tsc.features
y = tsc.data[tsc.name + '_scaled']

tscv = TimeSeriesSplit(n_splits=10, gap=1)
model = XGBRegressor(learning_rate=.1)#, booster='gbtree', n_estimators=1000)
model.fit(X, y)

renner_br_ac_1 - data:(275, 2) - features:(275, 8)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [ ]:
#gera as predicoes para o conjunto de teste
tsc = test_series['renner_br_ac_1']
X_test = tsc.features
y_test = tsc.data[tsc.name]

y_pred = model.predict(X_test)
print(y_pred)

fig = plt.figure(figsize=(15,8))
plt.plot(X_test, label='y_test')
plt.plot(pd.DataFrame(y_pred).rolling(4).mean(), label='prediction')
plt.legend()
plt.show()

In [63]:
tsc.features[['MA_3', 'MA_7', 'MA_14', 'MA_21', 'STD_3', 'STD_7', 'STD_14', 'STD_21']]

,MA_3,MA_7,MA_14,MA_21,STD_3,STD_7,STD_14,STD_21
ds,,,,,,,,
2022-04-10,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
2022-04-17,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
2022-04-24,0.883085,0.000000,0.0,0.0,0.734942,0.00000,0.0,0.0
2022-05-01,0.910448,0.000000,0.0,0.0,0.923784,0.00000,0.0,0.0
2022-05-08,1.000000,0.000000,0.0,0.0,0.725399,0.00000,0.0,0.0
2022-05-15,0.957711,0.000000,0.0,0.0,1.000000,0.00000,0.0,0.0
2022-05-22,0.825871,1.000000,0.0,0.0,0.685683,1.00000,0.0,0.0
2022-05-29,0.786070,0.969194,0.0,0.0,0.423485,0.89863,0.0,0.0


In [55]:

test_data = series_to_supervised(train_scaled[['renner_br_al_25']])
X_test = test_data[:, :-1]

row = X_test[:].flatten()

predictions = model.predict(np.asarray(row))

fig = plt.figure(figsize=(15,8))
plt.plot(X_test, label='y_test')
plt.plot(pd.DataFrame(predictions).rolling(4).mean(), label='prediction')
plt.legend()
plt.show()

XGBoostError: [18:15:41] C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/predictor/cpu_predictor.cc:377: Check failed: m->NumColumns() == model.learner_model_param->num_feature (1 vs. 8) : Number of columns in data must equal to trained model.

In [ ]:
data = series_to_supervised(train_scaled[['renner_ar_ex_1']])

X = data[:, :-1]
y = data[:, -1]

tscv = TimeSeriesSplit(n_splits=10, gap=1)

model = XGBRegressor(learning_rate=.1, booster='gbtree', n_estimators=1000)

scores = cross_val_score(model, X, y, cv=tscv, scoring='neg_mean_squared_error')
print(f'RMSE: {np.sqrt(-scores).mean()}')

In [ ]:
model = XGBRegressor(learning_rate=.1)
model.fit(X, y)


In [ ]:
test_data = series_to_supervised(test_scaled[['renner_br_al_25']])
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

row = X_test[:].flatten()

y_pred = model.predict(np.asarray(row))

acc = model.score(y_test, y_pred)
print(f'Accuracy: {acc.mean()}')


In [ ]:
train_data = series_to_supervised(train_scaled[['renner_ar_ex_1']])
test_data = series_to_supervised(test_scaled[['renner_ar_ex_1']])

X = train_data[:, :-1]
y = train_data[:, -1]

X_test = test_data[:, :-1]
y_test = test_data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X = train_scaled['renner_ar_ex_1'].values
y = train_scaled['renner_ar_ex_1'].shift(1).values

In [19]:

test_data = series_to_supervised(train_scaled[['renner_br_al_25']])
X_test = test_data[:, :-1]

row = X_test[:].flatten()

yhat = model.predict(np.asarray(row))

fig = plt.figure(figsize=(15,8))
plt.plot(X_test, label='y_test')
plt.plot(pd.DataFrame(yhat).rolling(4).mean(), label='yhat')
plt.legend()
plt.show()

XGBoostError: [16:48:32] C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-03de431ba26204c4d-1/xgboost/xgboost-ci-windows/src/predictor/cpu_predictor.cc:377: Check failed: m->NumColumns() == model.learner_model_param->num_feature (1 vs. 8) : Number of columns in data must equal to trained model.

In [ ]:
fig = sm.tsa.seasonal_decompose(pd.DataFrame(yhat, index=train_scaled[['renner_br_am_1']].index[1:])).plot()
fig.set_figheight(8)
fig.set_figwidth(15)


In [ ]:
train_scaled[['renner_br_al_25']]

In [ ]:
pd.DataFrame(yhat).rolling(14).mean()

In [ ]:
train_scaled

In [ ]:
train_scaled['renner_ar_ex_3'].values

In [ ]:
train.plot(legend=False, figsize=(15,8))

In [ ]:
data = train.copy(deep=True)
data.set_index('ds', inplace=True)
data[(data['tipo_loja'] == 'física') & (data['loja'] == 'renner_ar_ex_1')][['venda']].plot()

In [ ]:
train.groupby(by='loja').sum()

In [ ]:
train.pivot(index='ds', columns='loja', values='venda').fillna(0)

In [ ]:
temp = adjust_data(train)
temp

In [ ]:
column = 'renner_ar_ex_1'
tsData = TSData(column, temp)
tsData.features


In [ ]:
print(tsData)

In [ ]:
javascript:__doPostBack('ViewerControl1$TreeViewFiles','sR28324\\F2222850')

In [ ]:
temp = adjust_data(test)

scaler = MinMaxScaler()
data = scaler.fit_transform(temp['renner_br_rj_81'].values.reshape(-1,1))
data.reshape(1, -1)[0]

In [ ]:
temp[['renner_br_rj_81']].rolling(21).mean()

In [ ]:
temp = adjust_data(train)
temp['renner_br_rj_81']

tsData = TSData('renner_br_rj_81', temp)

In [ ]:
tsData.create_features(temp, 'renner_br_rj_81')

In [ ]:
#construir rede neural que recebe como entrada o conjunto de k dias anteriores e retorna o valor previsto para o dia seguinte
def build_model():
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    return model

In [ ]:
############################################################################################################

In [ ]:
temp = adjust_data(test)
#transform_data(temp, 'renner_ar_ex_2')
#temp = test.pivot(index='ds', columns='loja', values='venda').fillna(0).rename_axis(None, axis=1)

In [ ]:
fisica, web = split_data(train)

In [ ]:
web['loja'].unique()

In [ ]:
train[train['loja'].isin(['renner_br_sp_192', 'renner_br_rj_81'])]['tipo_loja'].unique()

In [ ]:
############################################################################################################

In [ ]:
temp = test.pivot(index='ds', columns='loja', values='venda').fillna(0)
temp


column = 'renner_br_ac_1'
temp[[column]]